#### Hecho por el grupo G08 con participantes Mauro Diaz Lupone, Adrian Skaczylo

### REPRESENTACIÓN
Antes de entrar en la implementación y la resolución del problema explicaremos la representación de la solución del problema

Se trata de un juego por turnos, en el que cada turno/segundo el jugador tiene dos posibles opciones, comprar minas o no hacer nada, por lo que hemos decido que el cromosoma represente el numero de minas de cada tipo que compra en cada segundo. Si n es el numero de segundos del que dispondrá el juego, el cromosoma estará formado por n genes. Cada gen tendrá m alelos siendo m el numero distinto de minas que hay disponibles en el juego. El alelo i-ésimo de un gen será un numero >= 0 que representa la cantidad de minas compradas del tipo i en el segundo que representa ese gen.
Suponiendo que hay 6 tipos de minas y la duracion del juego es de 900 segundos, implicará que el cromosoma tendrá una longitud de 900*6=5400 digitos. De esta forma cada subsecuencia de 6 digitos representará la acción del jugador en ese segundo. Por ejemplo:

(000000 100100 000000 200100 ........ 110100)

Vemos que en el segundo 0 el jugador no ha comprado nada; en el segundo 1 el jugador ha comprado una mina del tipo 1 y otra del 4; en el siguiente turno no ha comprado nada; en el segundo 3 ha comprado 3 minas, dos del tipo 1 y otra del tipo 4; y asi sucesivamente con los restantes genees que representan el resto de los turnos

### IMPLEMENTACIÓN DEL PROBLEMA
Siguiendo el esquema de lo visto en Geneticos1, hemos decidio implementar la clase IddleGame con las funciones necesarias para la resolucion del problema genetico. La inicializacion de un objeto de esta clase necesitará tres argumentos nuevos de los que tenía la clase "ProblemaGenetico": una lista con los costes de cada mina (costeMina),una lista con las mejoras que aporta cada mina comprada (gananciaMina) y los segundos de la partida.

In [5]:
import random

In [6]:
class IddleGame(object):
        def __init__(self, fun_dec,fun_muta , fun_cruza, fun_fitness, segundos, costeMina, gananciaMina):
            
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            
            self.longitud_individuos = segundos*len(costeMina)      #longitud cromosoma
            self.costeMina = costeMina                              #lista con los costes de cada mina
            self.gananciaMina = gananciaMina                        #lista con las mejoras por segundo de cada 
            self.segundos =segundos                                 #segundos que va a durar la partida
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo, self.costeMina, self.gananciaMina,self.segundos)
            return fenotipo
        
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma, self.costeMina, self.gananciaMina,self.segundos)
            return valoracion

### Decodificar
Como con esta representación de los crmosomas no todos pueden ser válidos, aplicaremos la misma técnica que en el problema de la mochila.
La funcion decodificar se va a encargar de hacer válidos todos los cromosomas de la siguiente manera:
Cada gen representa las minas compradas en ese cierto segundo, entonces en caso de que el jugador haya comprado minas que no puede costear, anularemos dichas compras. Si en un cierto segundo quiere comprar x minas de tipo i pero solo puede permitirse y < x minas de tipo i, se anulará toda la compra de ese tipo de minas en ese segundo.

Será necesario pasar como argumentos costeMina y gananciaMina para poder evaluar que gen es valido y cual no.

In [7]:
def decodifica(cromosoma, costeMina, gananciaMina,segundos):
    numMinas = len(costeMina)
    l = [0] * len(cromosoma) #lista inicializada a 0 de longitud cromosoma
    oroTotal = 0
    oroXsegundo = 1
    for i in range(0, segundos):
        genActual = cromosoma[numMinas * i : numMinas * (i + 1)] #parte del cromosoma que corresponde al segundo i
        coste, mejoraOro = 0, 0
        for j in range(0, numMinas):
            coste = genActual[j] * costeMina[j]
            mejoraOro = genActual[j] * gananciaMina[j]
            if coste > 0 and oroTotal >= coste:
                oroXsegundo += mejoraOro
                oroTotal -= coste
                l[j +i*numMinas] = genActual[j]
        oroTotal += oroXsegundo
    return l

Veamos unos ejemplos. Vamos poner que hay tres tipos de minas con costes (1,2,3) y que cada una de ellas aumenta las ganancias por segundo en (1,2,3) respectivamente.

In [5]:
#Es un cromosoma valido, pues en el segundo 1 hace una compra y tiene dinero para ello
decodifica((0,0,0,1,0,0,0,0,0),[1,2,3],[1,2,3],3)

[0, 0, 0, 1, 0, 0, 0, 0, 0]

In [6]:
#No es un cromosoma valido pues en el instante 0 no tiene dinero y sin embargo ha hecho una compra
#Anularemos la compra
decodifica((1,0,0,1,0,0,0,0,0),[1,2,3],[1,2,3],3)

[0, 0, 0, 1, 0, 0, 0, 0, 0]

In [7]:
#Compra 1000 minas en el segundo 2 => anulará toda la compra
decodifica((1,0,0,1,0,0,1000,1,0),[1,2,3],[1,2,3],3)

[0, 0, 0, 1, 0, 0, 0, 1, 0]

En este caso la compra de la mina tipo2  en el ultimo segundo no la ha anulado pues si era posible esa acción. Solo anula el total de la compra de un tipo de mina si no es posible, es decir, o compra todas las minas de un mismo tipo o no compra ninguna

### Fitness
La funcion fitness calculará el oro total que tiene el jugador tras acabar la partida. Para ello necesitará ver la partida que representa un cromosoma con la codificación que hemos implementado. Se asume que el cromosoma es válido, porque será antes decodificado.

Al igual que antes, necesitará como argumentos costeMina, gananciaMina y los segundos

In [8]:
def fun_fitness(cromosoma, costeMina, gananciaMina,segundos):
    
    numMinas = len(costeMina)
    l = decodifica(cromosoma, costeMina, gananciaMina,segundos)
    
    ceros = [0] * numMinas #variable auxiliar 
    oroTotal= 0
    oroXsegundo=1
    
    for i in range(segundos):
        segundoActual = l[numMinas * i : numMinas * (i + 1)] #lista de compras realizadas en el segundo i
        if segundoActual != ceros:
            for j in range(numMinas):
                oroTotal -= segundoActual[j] * costeMina[j]
                oroXsegundo += segundoActual[j] * gananciaMina[j]
        oroTotal += oroXsegundo
    return oroTotal

In [9]:
# Segundo 0 : no compra nada  y recibe uno de oro => oroTotal = 1
#Segundo 1 :  compra una mina de coste 1 y recibe 2 de oro tras finalizar el segundo => oroTotal = 2
#Segundo 2:  no compra nada y recibe dos de oro => oroTotal = 4

fun_fitness((0,0,0,1,0,0,0,0,0),[1,2,3],[1,2,3],3)

4

In [10]:
#Tras decodificar en la funcion fitness estamos en el ejemplo anterior
fun_fitness((1,0,0,1,0,0,0,0,0),[1,2,3],[1,2,3],3)

4

In [11]:
#Decodifica el cromosoma en (000 100 010)
#Segundo 0: recibe uno de oro tras finalizar el segundo  => oroTotal = 1
#Segundo 1: compra una mina de coste 1 y tras finalizar el segundo recibe 2 de oro => oroTotal=2
#Segundo 3 : compra una mina de coste 2 y tras finalizar el segundo recibe 2+2 de oro => oroTotal = 4
fun_fitness((1,0,0,1,0,0,1000,1,0),[1,2,3],[1,2,3],3)

4

### Cruzar y Mutar

Realizamos un cruce de dos puntos.
Para la funcion mutar se elige un digito aleatorio del cromosoma y si se elige al final mutar se le suma o resta 1 de forma aleatoria (la probabilidad de que se sume es la misma de la de que se reste). Si se elige mutar y el digito aleatorio del cromosoma es 0 (luego no ha comprado ninguna mina en ese segundo de ese tipo) se le sumará automaticamente 1 (ya que no tiene sentido restar).

In [9]:
def fun_cruzar(cromosoma1, cromosoma2):
    """Cruce de dos puntos"""
    n = len(cromosoma1)
    n = n // 3
    cruce1 = cromosoma1[0:n]+cromosoma2[n:2 * n]+cromosoma1[2 * n: len(cromosoma1)]
    cruce2 = cromosoma2[0:n]+cromosoma1[n:2 * n]+cromosoma2[2 * n: len(cromosoma1)]
    return [cruce1,cruce2]

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        if cromosoma[p] > 0:
            if random.uniform(0,1) >= 0.5:
                cromosoma[p] += 1
            else:
                cromosoma[p] -= 1
        else:
            cromosoma[p] += 1
    return cromosoma

### Poblacion Inicial

Procedemos a implementar la funcion que nos dará la población inicial. Durante el trabajo se han pensado tres implementaciones distintas de las cuales luego experimentando se concluirá cual es la mejor.

En la primera  los individuos solo compran entre 0 y 2 minas de cada tipo. Esto era con el objetivo de crear poblaciones validas desde un principio. Nos dimos cuenta más tarde que no era lo mejor porque generalmente más pasa el tiempo más poder adquisitivo tiene el jugador y puede comprar un mayor número de minas. Dejamos la implementación aunque no se utilizará porque en los experimentos siempre nos ha dado soluciones peores y tiempos de ejecución mayores.

En la segunda, la primera mitad del cromosoma esta formado por numeros entre 0 y 2, y en la otra mitad por numeros entre 0 y 5 para dar así mas variedad a la poblacion. La asignacion de estos numeros para ciertos alelos se hará siempre de forma aleatoria.

La tercera es igual que la segunda pero la segunda mitad del cromosoma tendrá números más grandes (entre 0 y 10000).

In [18]:
def poblacion_inicial1(problema_genetico, size):
    l=[] # población inicial
    for i in range(size): # añadimos a la población size individuos
        x = [0] * problema_genetico.longitud_individuos
        for _ in range(2 * problema_genetico.longitud_individuos // 3):  
            p = random.randint(0,problema_genetico.longitud_individuos - 1)
            x[p] = random.randint(0, 2)
        l.append(x) 
    return l

In [14]:
#Al principio compra menos minas y de mitad hacia adelante el numero de minas que compra es mayor
def poblacion_inicial2(problema_genetico, size):
    l=[] # población inicial
    for i in range(size): # añadimos a la población size individuos
        x = [0] * problema_genetico.longitud_individuos
        for _ in range(problema_genetico.longitud_individuos // 3):                                      
            p = random.randint(0,problema_genetico.longitud_individuos - 1)
            x[p] = random.randint(0, 2)
    
        for _ in range(problema_genetico.longitud_individuos // 6):
            p = random.randint(problema_genetico.longitud_individuos//2,problema_genetico.longitud_individuos - 1)
            x[p] = random.randint(0,5)
        
        l.append(x)
    return l

In [15]:
def poblacion_inicial3(problema_genetico, size):
    l=[] # población inicial
    for i in range(size): # añadimos a la población size individuos
        x = [0] * problema_genetico.longitud_individuos
        for _ in range(5 * problema_genetico.longitud_individuos // 12):                                      
            p = random.randint(0,problema_genetico.longitud_individuos - 1)
            x[p] = random.randint(0, 3)
    
        for _ in range(5 * problema_genetico.longitud_individuos // 12):
            p = random.randint(problema_genetico.longitud_individuos//2,problema_genetico.longitud_individuos - 1)
            x[p] = random.randint(0,10000)
        
        l.append(x)
    return l

Para los siguientes ejemplos se tratará de una partida de 4 segundos en la que el jugador puede comprar 6 tipos de mina, es decir, cromosomas de 4*6 de longitud

In [16]:
partida = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,4,[1,2,3,4,5,6],[1,2,3,4,5,6])

In [19]:
#Generaremos una poblacion de 10 individuos
poblacion_inicial1(partida,10)

[[2, 1, 2, 2, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2],
 [0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 2, 0],
 [0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 0, 2, 1, 1, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0],
 [1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2],
 [1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]]

In [21]:
poblacion_inicial2(partida,10)

[[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 3, 0, 3, 0, 0, 2, 4, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 0, 5, 0, 0],
 [1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 4, 0, 5, 3, 0, 0],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 5, 0, 0, 0, 0, 3],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 1, 1, 2, 0, 2],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 3, 2, 4, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0],
 [0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 3, 5, 0, 0, 0, 0, 1]]

In [22]:
poblacion_inicial3(partida,10)

[[1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  3,
  2,
  0,
  6612,
  1842,
  8254,
  0,
  945,
  1,
  6788,
  0,
  7159,
  0,
  0,
  2490],
 [0,
  0,
  0,
  3,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  8352,
  0,
  8036,
  5963,
  241,
  3,
  7156,
  0,
  0,
  3671,
  7651,
  8978],
 [0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  4117,
  1290,
  0,
  6559,
  0,
  6294,
  4654,
  5424,
  0,
  2,
  7087],
 [2,
  0,
  2,
  3,
  3,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  4873,
  8332,
  0,
  9101,
  2850,
  3246,
  2163,
  1,
  0,
  517,
  2497,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  9261,
  7695,
  4203,
  8953,
  3,
  1,
  1,
  5842,
  4327,
  0,
  5704,
  2],
 [0,
  0,
  0,
  3,
  0,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  9425,
  5004,
  0,
  0,
  6712,
  1133,
  6756,
  3325,
  2,
  5810],
 [2,
  0,
  1,
  1,
  1,
  0,
  0,
  2,
  3,
  0,
  0,
  0,
  3432,
  8331,
  0,
  0,
  3098,
  5390,
  0,
  0,
  312,
  5453,
  3,
  9629],
 [0,

### Selección De Individuos
La selección de los invidividuos que se van a reproducir la haremos por torneo.
Se recibirá una instancia de IddleGame, una poblacón y seleccionará n individuos donde cada uno sale de un torneo de k que participantes.

In [10]:

def seleccion_por_torneo(problema_genetico, poblacion, n, k):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes"""
    seleccionados = []
    for i in range(n): #Se realizan n torneos
        participantes = random.sample(poblacion,k)  #Elegimos k participantes
        seleccionado = max(participantes, key=problema_genetico.fitness) #Elegimos al mejor
        max(poblacion, key=problema_genetico.fitness) #¿? esto de aqui que hace
        seleccionados.append(seleccionado)
        
    return seleccionados

Veamos un par de ejemplos

In [24]:
partida2 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,4,[1,2,3],[1,2,3])

In [25]:
poblacion1= poblacion_inicial1(partida2,17)
poblacion1

[[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1],
 [2, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1],
 [0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 1],
 [2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 1, 2, 0, 2, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0],
 [0, 2, 0, 2, 1, 2, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0]]

In [26]:
seleccion_por_torneo(partida2,poblacion1,3,5)

[[0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0],
 [2, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 1]]

In [27]:
seleccion_por_torneo(partida2,poblacion1,3,5)

[[0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0],
 [0, 0, 0, 1, 2, 0, 2, 1, 0, 1, 0, 0],
 [0, 0, 0, 1, 2, 0, 2, 1, 0, 1, 0, 0]]

### Cruce  Y Mutación De Individuos Seleccionados

Una vez tenemos seleccionados los individuos es hora de que se reproduzcan y posteriormente muten si es el caso. Realizaremos un cruce no destructivo, de esta manera aseguraremos que siempre pasan a la siguiente generacion los mejores.

In [11]:
#Entre los dos padres y los dos hijos, pasan los mejores
def cruza_padres_NO_destructiva(problema_genetico,padres):
    l = []
    for i in range(len(padres)//2):
        #El mejor
        hijos = problema_genetico.fun_cruza(padres[2*i],padres[2*i+1]) 
        familia = [hijos[0], hijos[1], padres[2*i], padres[2*i+1]] 
        mejor = max(familia, key=problema_genetico.fitness) 
        familia.remove(mejor)
        #El segundo mejor                         
        segmejor = max(familia, key=problema_genetico.fitness)
        l.append(mejor) 
        l.append(segmejor)
    return l

In [12]:
def muta_individuos(problema_genetico, poblacion, prob):
    l = []
    for i in poblacion:
        l.append(problema_genetico.muta(i,prob))
    return l

### Nueva Generación
Ya tenemos todos los ingredientes para la crear las nuevas generaciones


In [13]:
def nueva_generacion(problema_genetico, k, poblacion, numPadres, numNoPadres, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, numNoPadres, k) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, numPadres , k)
    cruces =  cruza_padres_NO_destructiva(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

In [32]:
partida3 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,4,[1,2,3],[1,2,3])
poblacion3= poblacion_inicial1(partida2,5)
poblacion3

[[0, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0],
 [0, 1, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0],
 [0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [33]:
generacion_de_poblacion3 = nueva_generacion(partida3,3,poblacion3,2,3,0.2)
generacion_de_poblacion3

[[0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 [0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 [0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 [0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0]]

Veamos si esta generacion es mejor que la incial. Para ello calculemos el fitness total de cada poblacion. Comenzemos con el de la poblacion incial

In [35]:
fitnessTotal = 0
for individuo in poblacion3:
    fitnessTotal +=partida3.fitness(individuo)
fitnessTotal # =24

24

Y ahora el de la nueva generación

In [37]:
fitnessTotalNuevaGen = 0
for individuo in generacion_de_poblacion3:
    fitnessTotalNuevaGen +=partida3.fitness(individuo)

fitnessTotalNuevaGen # = 30

30

Efectivamente, la nueva generación es mejor debido al cruce no destructivo

### Algoritmo Genetico
Finalmente,implementamos el algoritmo genetico que nos permitirá resolver el problema. Implementaremos dos distintos, uno de ellos genera la población inicial con el algoritmo numero 2 implementado, mientras el otro lo genera con el 3er algoritmo. No se implementa uno para el 1er algoritmo de población inicial porque en los experimentos previos la diferencia con los demás era tan grande que no es interesante dejarlo por escrito.

In [14]:
# Sus argumentos son:
# k: número de participantes que se seleccionaran en cada torneo de seleccion
# numGen: número de generaciones (que se usa como condición de terminación)
# numNuevaGeneracion: número de individuos en cada generación
# prop_cruce: proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.


#Genera la poblacion inicial con poblacion_incial2
def algoritmo_genetico(problema_genetico,k,numGen,numNuevaGeneracion,prop_cruces,prob_mutar):
    
    #Creamos la poblacion inicial
    poblacion= poblacion_inicial2(problema_genetico, numNuevaGeneracion)
    
    numPadres=round(numNuevaGeneracion*prop_cruces)
    numPadres= int (numPadres if numPadres%2==0 else numPadres-1) #Haremos que el numero de padres sea par
    
    numNoPadres = numNuevaGeneracion-numPadres #numero de personas que no se cruzaran
    
    for _ in range(numGen):
        poblacion= nueva_generacion(problema_genetico,k, poblacion,numPadres, numNoPadres,prob_mutar)

    mejor_cr= max(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 


#Genera la poblacion inicial con poblacion_incial3
def algoritmo_genetico3(problema_genetico,k,numGen,numNuevaGeneracion,prop_cruces,prob_mutar):
    
    #Creamos la poblacion inicial
    poblacion= poblacion_inicial3(problema_genetico, numNuevaGeneracion)
    
    numPadres=round(numNuevaGeneracion*prop_cruces)
    numPadres= int (numPadres if numPadres%2==0 else numPadres-1) #Haremos que el numero de padres sea par
    
    numNoPadres = numNuevaGeneracion-numPadres #numero de personas que no se cruzaran
    
    for _ in range(numGen):
        poblacion= nueva_generacion(problema_genetico,k, poblacion,numPadres, numNoPadres,prob_mutar)

    mejor_cr= max(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 



### PRUEBAS
Vamos a poner en marcha el algoritmo. Para ello primero implementamos una función que permita mostrarnos la informacion necesaria de la solucion.

In [15]:
def mostrarResultado(solucion,segundos,costeMina,gananciaMina):
    numTipoMinas = len(gananciaMina)
    print("Total de ganancias: "+ str(solucion[1]))
    
    OroTotal = 0
    oroXsegundo = 1
    for i in range(segundos):
        print("En el segundo "+str(i)+ " comienza con "+str(OroTotal) + " de oro y tiene "+str(oroXsegundo)+" oro por segundo")
        comprasActual = solucion[0][i*numTipoMinas : numTipoMinas +i*numTipoMinas]
        
        for j in range(numTipoMinas):
            mejoraOroXsegundo = 0
            if comprasActual[j] >0:
                mejoraOroXsegundo+=gananciaMina[j]
                oroXsegundo+=gananciaMina[j]
                OroTotal -=costeMina[j]
                print("         -Ha comprado "+str(comprasActual[j]) +" minas de tipo "+str(j))
        
        OroTotal+=oroXsegundo

### Prueba 1
- 30 segundos

- Poblaciones  de 20 individuos

- 100 generaciones

- Poblacion inicial generado con poblacion_inicial2


In [41]:
problema = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,30,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

In [42]:
solucion = algoritmo_genetico(problema,5,100,20,8,0.4)

In [43]:
mostrarResultado(solucion,30,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 30
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En el s

Observamos que en esta solución se ha optado por comprar una mina en el segundo 15. Para estos segundos en verdad hay otra solución óptima, que es no hacer nada, ya que conseguiriamos tener las mismas monedas. Que salga la solución de comprar mina depende de como se cree la población inicial y si se da el caso que en el cruce o mutación aparezca.

Datos Extra:

-Tiempo de compilacion 1min 23 s

### Prueba 2
- 60 segundos

- Poblacion de 20 individuos

- 100 generaciones

- Poblacion inicial generado con poblacion_inicial2

In [44]:
problema2 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,60,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion2 = algoritmo_genetico(problema2,5,100,20,8,0.4)
#4 mins

In [45]:
mostrarResultado(solucion2,60,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 144
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En el 

En esta prueba la mejor solución parece ser la compra de la mina mas barata siempre que se pueda. Además el jugador deja de comprar minas en los ultimos segundos para no gastar más dinero del que puede ganar.

### Prueba 3
- 90 segundos

- Poblacion de 20 individuos

- 100 generaciones

- Poblacion inicial generado con poblacion_inicial2

In [46]:
problema3 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,90,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))


In [47]:
solucion3 = algoritmo_genetico(problema3,5,100,20,8,0.4)
#8 mins

In [48]:
mostrarResultado(solucion3,90,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 713
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En el 

Obsevamos que el jugador compra minas segun va obteniendo dinero. No es la solución óptima, podría no haber comprado las últimas minas. Conque haga m´s iteraciones bastaría.

### Prueba 4

- 300 segundos
- Poblacion de 20 individuos
- 100 generaciones
- Poblacion inicial generado con poblacion_inicial2

In [49]:
problema4 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,300,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion4 = algoritmo_genetico(problema4,5,100,20,8,0.4)
#tiempo 20 mins

In [50]:
mostrarResultado(solucion4,300,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 6688298
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En

En esta situación,la ultima acción no sería la correcta pues realiza una compra en el ultimo turno de la partida, lo cual no hace que sea la mas optima pues el coste de la mina es mayor que el beneficio que da. Aun así, teniendo en cuenta que el jugador llega al último turno con 190497 de oro por segundo y el coste de estas ultima mina es 15 + 1400, implica que la diferencia va a ser minima respecto a la solución minima. Seguramente,  si hubiesemos hecho mas generaciones, se hubiese mejorado este cromosoma.

### Prueba 5

- 900 segundos
- Poblacion de 20 individuos
- 50 generaciones
- Poblacion inicial generado con poblacion_inicial2

In [51]:
problema5 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion5 = algoritmo_genetico(problema5,5,50,20,8,0.4)
#tiempo 37 min

In [52]:
mostrarResultado(solucion5,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 1575789410
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo

En este caso, el comportamiento del jugador es bastante bueno, pues realiza muchas compras a lo largo de la partida, y según se va a acercando al final reduce considerablemente la compra de minas (aunque en el ultimo segundo haga unas compras).

### Prueba 6

- 30 segundos
- Poblacion de 20 individuos
- 100 generaciones
- Poblacion inicial generado con la funcion poblacion_inicial3

In [53]:
### pruebas distintas poblaciones iniciales
problema6 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,30,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion6_2 = algoritmo_genetico3(problema6,5,100,20,8,0.4)

In [54]:
mostrarResultado(solucion6_2,30,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 30
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En el s

El comportamiento es perfecto, igual que la prueba 1

### Prueba 7

- 90 segundos
- Poblacion de 20 individuos
- 100 generaciones
- Poblacion inicial generado con la funcion poblacion_inicial3

In [55]:
problema7 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,90,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion7_1 = algoritmo_genetico3(problema7,5,100,20,8,0.4)
#15 mins

In [56]:
mostrarResultado(solucion7_1,90,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))


Total de ganancias: 918
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
En el 

Comparando este resultado con Prueba 3 vemos que la solución es mejor y que aquí el jugador compra mas minas. Esto idnica ya que la creacion de la población inicial ha influido para bien ya que no tenemos gran cantidad de iteraciones y cuanto mejor sea el inicio mejor será el final. Si hubiera habido más iteraciones esto tendría menos importancia ya que habría mas oportunidades de encontrar la mejor solucion.

### Prueba 8

- 300 segundos
- Poblacion de 20 individuos
- 100 generaciones
- Poblacion inicial generado con la funcion poblacion_inicial3

In [57]:
problema8= IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,300,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion8 = algoritmo_genetico3(problema8,5,100,20,8,0.4)
#25 min

In [58]:
mostrarResultado(solucion8,300,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 11574410
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segundo
E

La solución en este caso es mucho mejor que la del problema 4 : 11574410 >> 6688298. Y esto es gracias a que al tratarse de una partida mucho mas larga, y una generación de cromosomas con numeros mas grandes, se generarán partidas en el que la compra de minas es bastante mayor con respecto a los otras pruebas. Eso sí, en el ultimo segundo hace una gran compra que reduce el dinero total ganado, esto es el problema de usar este generador de poblacion inicial. Vemos aún así que encuentra una solución mejor.

### Prueba 9

- 900 segundos
- Poblacion de 20 individuos
- 50 generaciones
- Poblacion inicial generado con la funcion poblacion_inicial3

In [59]:
problema9 = IddleGame(decodifica,fun_mutar,fun_cruzar,fun_fitness,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion9 = algoritmo_genetico3(problema9,5,50,20,8,0.4)
#30 min

In [60]:
mostrarResultado(solucion9,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 1682669273421
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segu

Al igual que antes, si comparamos esta prueba con Prueba 5 obtenemos un mejor resultado: 1.682.669.273.421 >> 1.575.789.410. Esto es debido a que compra muchisimas mas minas durante la partida, y segun se acerca el final se reducen dichas compras. Al mismo tiempo el tener más cromosomas la población con compra grande de minas es probable que alguna acierte y mejore la solcuión óptima. Nos encontramos el mismo problema de que compra gran cantidad de minas en los ultimos segundos, aunque aun así el hacer como mejor esrategia comprar muchas minas vemos que sale mejor.

## Experimento final

Después de analizar los resultados de los experimentos anteriores, se ha decidido intentar mejorar nuestro algoritmo genetico. Para ello hemos querido crear cromosomas en los que en la parte inicial y la parte inicial no se compran tantas minas y en la parte del medio compre muchas. Esto es justo porque en la primera parte el jugador no tendrá mucho poder adquisitivo y en los ultimos segundos comprar minas grandes repercutirá a peor. Por ello la población inicial se creará de esa manera, considerando la parte inicial el primer tercio del cromosoma y la parte final el ultimo 1/15 del cromosoma. Se cambia también la función mutacion para que mute con un poco mas de variedad y se prueba directamente para 900 segundos igual que en la prueba 9.

In [17]:
def fun_mutar4(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        r = random.randint(0, 3)
        if cromosoma[p] > r:
            if random.uniform(0,1) >= 0.5:
                cromosoma[p] += r
            else:
                cromosoma[p] -= r
        else:
            cromosoma[p] += r
    return cromosoma

def poblacion_inicial4(problema_genetico, size):
    l=[] # población inicial
    for i in range(size): # añadimos a la población size individuos
        x = [0] * problema_genetico.longitud_individuos
        for _ in range(3 * problema_genetico.longitud_individuos // 4):                                      
            p = random.randint(0,problema_genetico.longitud_individuos - 1)
            if (p in range(0, problema_genetico.longitud_individuos // 3) or 
                p in range(14 * problema_genetico.longitud_individuos // 15, problema_genetico.longitud_individuos)):
                x[p] = random.randint(0, 3)
            else:
                x[p] = random.randint(0,10000)

        l.append(x)
    return l

In [18]:
#Genera la poblacion inicial con poblacion_incial3
def algoritmo_genetico4(problema_genetico,k,numGen,numNuevaGeneracion,prop_cruces,prob_mutar):
    
    #Creamos la poblacion inicial
    poblacion= poblacion_inicial4(problema_genetico, numNuevaGeneracion)
    
    numPadres=round(numNuevaGeneracion*prop_cruces)
    numPadres= int (numPadres if numPadres%2==0 else numPadres-1) #Haremos que el numero de padres sea par
    
    numNoPadres = numNuevaGeneracion-numPadres #numero de personas que no se cruzaran
    
    for _ in range(numGen):
        poblacion= nueva_generacion(problema_genetico,k, poblacion,numPadres, numNoPadres,prob_mutar)

    mejor_cr= max(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 

In [19]:
problema10 = IddleGame(decodifica,fun_mutar4,fun_cruzar,fun_fitness,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))
solucion10 = algoritmo_genetico4(problema10,5,50,20,8,0.4)
# 40 mins

In [21]:
mostrarResultado(solucion10,900,(15,100,300,2000,15000,250000),(1,5,20,100,700,10000))

Total de ganancias: 2987275861333
En el segundo 0 comienza con 0 de oro y tiene 1 oro por segundo
En el segundo 1 comienza con 1 de oro y tiene 1 oro por segundo
En el segundo 2 comienza con 2 de oro y tiene 1 oro por segundo
En el segundo 3 comienza con 3 de oro y tiene 1 oro por segundo
En el segundo 4 comienza con 4 de oro y tiene 1 oro por segundo
En el segundo 5 comienza con 5 de oro y tiene 1 oro por segundo
En el segundo 6 comienza con 6 de oro y tiene 1 oro por segundo
En el segundo 7 comienza con 7 de oro y tiene 1 oro por segundo
En el segundo 8 comienza con 8 de oro y tiene 1 oro por segundo
En el segundo 9 comienza con 9 de oro y tiene 1 oro por segundo
En el segundo 10 comienza con 10 de oro y tiene 1 oro por segundo
En el segundo 11 comienza con 11 de oro y tiene 1 oro por segundo
En el segundo 12 comienza con 12 de oro y tiene 1 oro por segundo
En el segundo 13 comienza con 13 de oro y tiene 1 oro por segundo
En el segundo 14 comienza con 14 de oro y tiene 1 oro por segu

Nos fijamos que tenemos una mejor solución que en Prueba 9: 2.987.275.861.333 >> 1.682.669.273.421. Esto es justamente debido a esta forma mejorada de crear la población inicial. En esta solución se sigue comprando en los ultimos segundos, ya que para que no fuese así o tenemos mucha suerte con la población inicial creada o deberiamos iterar por mucho más tiempo. Lo bueno es que esta vez no se compran demasiadas minas en esos ultimos segundos lo que mejora la solución. El algoritmo se podría ir mejorando pero el final lo más decisivo para llegar a la solución óptima sería dejarlo mucho más tiempo ejecutando con muchas más iteraciones.

### MEJOR SOLUCIÓN

- 900 segundos
- torneo de 5 jugadores
- Poblacion de 20 individuos
- 50 generaciones
- Proporcion de cruce de 8
- Probabilidad de mutación de 0.4
- Poblacion inicial generado con la funcion poblacion_inicial4

Oro total de 2.987.275.861.333 en un tiempo de ejecución de 48 minutos.